In [1]:
from cloudsafe import *
from utils.folium_utils import get_geojson_grid
database = 'here-api/main.db'
conn = create_connection(database)
departures = select_all_departures(conn)
stations = select_all_stations(conn)
bus_stops = select_all_bus_stops()

In [2]:
coords = stations.iloc[0].coordinates_here.split(',')
m = folium.Map([coords[0], coords[1]], zoom_start=10)
stations[['latitude', 'longitude']] = stations.apply(get_coords, axis=1, result_type="expand")

In [3]:
frequencies = stations.name_overpass.value_counts()

In [4]:
frequencies

Vigerslevvej             9
Universitetsparken       7
Trianglen St.            7
Strandboulevarden        7
Emdrup Torv              6
                        ..
Nørreport Station        1
Otto Busses Vej          1
Urtehaven                1
Ålekistevej, Sløjfen     1
H. C. Andersens Blvd.    1
Name: name_overpass, Length: 458, dtype: int64

In [5]:
rows = []
for station in frequencies.keys():
    first = stations.loc[stations['name_overpass'] == station].iloc[0]
    rows.append({'station_name': station,
                 'longitude': first.longitude,
                 'latitude': first.latitude,
                 'bus_count': frequencies.get(station)})

final = pd.DataFrame(rows)
final['longitude'] = final['longitude'].astype(float)
final['latitude'] = final['latitude'].astype(float)
top_right = [final['latitude'].max(), final['longitude'].max()]
bottom_left = [final['latitude'].min(), final['longitude'].min()]

In [6]:
first

id                                                        270
id_overpass                                   node/1727658440
name_overpass                           H. C. Andersens Blvd.
coordinates_overpass                      55.680234,12.564097
id_here                                             205078044
name_here               Vester Farimagsgade (Gyldenløvesgade)
coordinates_here                          55.680168,12.562927
no_data                                                     0
duplicate                                                   0
latitude                                            55.680168
longitude                                           12.562927
Name: 269, dtype: object

In [7]:
rows

[{'station_name': 'Vigerslevvej',
  'longitude': '12.491016',
  'latitude': '55.670034',
  'bus_count': 9},
 {'station_name': 'Universitetsparken',
  'longitude': '12.558402',
  'latitude': '55.702868',
  'bus_count': 7},
 {'station_name': 'Trianglen St.',
  'longitude': '12.577824',
  'latitude': '55.699336',
  'bus_count': 7},
 {'station_name': 'Strandboulevarden',
  'longitude': '12.588667',
  'latitude': '55.703497',
  'bus_count': 7},
 {'station_name': 'Emdrup Torv',
  'longitude': '12.52994',
  'latitude': '55.723076',
  'bus_count': 6},
 {'station_name': 'Sundbyvester Plads',
  'longitude': None,
  'latitude': None,
  'bus_count': 6},
 {'station_name': 'Bellahøj',
  'longitude': '12.51633',
  'latitude': '55.705484',
  'bus_count': 6},
 {'station_name': 'Baltikavej',
  'longitude': '12.597485',
  'latitude': '55.71696',
  'bus_count': 6},
 {'station_name': 'Folehaven',
  'longitude': '12.489274',
  'latitude': '55.652622',
  'bus_count': 6},
 {'station_name': 'Toftegårds Plads',

In [8]:
final

,station_name,longitude,latitude,bus_count
0,Vigerslevvej,12.491016,55.670034,9
1,Universitetsparken,12.558402,55.702868,7
2,Trianglen St.,12.577824,55.699336,7
3,Strandboulevarden,12.588667,55.703497,7
4,Emdrup Torv,12.529940,55.723076,6
...,...,...,...,...
453,Nørreport Station,NaN,NaN,1
454,Otto Busses Vej,12.558478,55.661684,1
455,Urtehaven,NaN,NaN,1
456,"Ålekistevej, Sløjfen",12.480334,55.691441,1


In [9]:
# Generating a grid heatmap (utils/folium_utils.py)
grid = get_geojson_grid(top_right, bottom_left, n=15)

In [10]:
grid

[{'type': 'FeatureCollection',
  'properties': {'lower_left': [12.45777, 55.61798],
   'upper_right': [12.4691428, 55.625308000000004]},
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Polygon',
     'coordinates': [[[12.45777, 55.625308000000004],
       [12.4691428, 55.625308000000004],
       [12.4691428, 55.61798],
       [12.45777, 55.61798],
       [12.45777, 55.625308000000004]]]}}]},
 {'type': 'FeatureCollection',
  'properties': {'lower_left': [12.4691428, 55.61798],
   'upper_right': [12.4805156, 55.625308000000004]},
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Polygon',
     'coordinates': [[[12.4691428, 55.625308000000004],
       [12.4805156, 55.625308000000004],
       [12.4805156, 55.61798],
       [12.4691428, 55.61798],
       [12.4691428, 55.625308000000004]]]}}]},
 {'type': 'FeatureCollection',
  'properties': {'lower_left': [12.4805156, 55.61798],
   'upper_right': [12.4918884, 55.625308000000004]},
  'features': [{'type': 'Feature',
  

In [11]:
# Frequencies by latitude and longitude
counts = []
for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]
    items = final.loc[(final['latitude'] <= upper_right[1]) &
                      (final['longitude'] <= upper_right[0]) &
                      (final['longitude'] >= lower_left[0]) &
                      (final['latitude'] >= lower_left[1])]
    counts.append(len(items))
max_count = max(counts)

In [12]:
# Add GeoJson to map
for i, geo_json in enumerate(grid):
    color = plt.cm.Reds(counts[i] / max_count)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, 
                        color=color: {
                            'fillColor': color,
                            'color': "black",
                            'weight': 2,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.55,
                        })

    m.add_child(gj)

In [13]:
m